In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building a Conversational Search Agent with Reasoning Engine and RAG on Vertex AI Search

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_vertex_ai_search_rag_agent.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine%2Ftutorial_vertex_ai_search_rag_agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_vertex_ai_search_rag_agent.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tutorial_vertex_ai_search_rag_agent.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

### Reasoning Engine in Vertex AI

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service in Vertex AI that helps you to build and deploy an agent reasoning framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Reasoning Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Reasoning Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.

### Objectives

In this tutorial, you will build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.

Your agent will use [LangChain](https://python.langchain.com/docs/get_started/introduction) and [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction) to retrieve structured data indexed from the [Movies Dataset on Kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv) using retrieval augmented generation (RAG).

- Install the Vertex AI SDK for Python
- Define a model for your agent
- Define Python functions as tools so that our agent can:
   - Search and retrieve movie information from Vertex AI Search
- Use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine
- Test your agent locally before deploying
- Deploy and test your agent on Reasoning Engine in Vertex AI

### Enable APIs and Services

This tutorial uses the following billable components of Google Cloud, which you'll need to enable for this tutorial:

- [Enable Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com/overview)
- [Enable Vertex AI Search API](https://console.cloud.google.com/apis/api/discoveryengine.googleapis.com/overview)
- [Enable Resource Manager API](https://console.cloud.google.com/apis/api/cloudresourcemanager.googleapis.com/overview)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python and extra dependencies related to Reasoning Engine, LangChain, and Vertex AI Search:

In [1]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    langchain-google-community \
    google-cloud-discoveryengine \
    google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 732.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [15]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "llmcoppelprueba"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://bucketfordecs"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Example: Building and Deploying a Conversational Search Agent for Movies

In this tutorial, you'll build Python functions as tools that the Gemini model will use to search for information about movies and ground its responses using a RAG implementation with Vertex AI Search.

### Import libraries

In [2]:
from IPython.display import Markdown, display
from googleapiclient import discovery
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.memory import ChatMessageHistory
from langchain_core import prompts
from vertexai.preview import reasoning_engines

### Define generative model

The first component of your agent involves the version of the generative model you want to use in your agent. Here you'll use the Gemini 1.5 Pro model:

In [3]:
model = "gemini-1.5-flash"

### Create a data store in Vertex AI Search

Now you'll create a data store within Vertex AI Search and index records from a movie data set.

Follow the tutorial steps in the Vertex AI Search documentation to [create a data store with structured data](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#structured-data), then [create a search app](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#structured-data_1) that points to that data store.

You should also enable the **Enterprise edition features** and **Advanced LLM features** options so that you can use the [LangChain retriever for Vertex AI Search](https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search/) within your agent to search indexed records.

Once the import is complete, you can navigate to your data store's **Data** page and obtain the values of your data store ID and region, which you can paste in the cell below:

<img width="60%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/reasoning-engine/images/vertex-ai-search-data-store-id.png">

In [4]:
DATA_STORE_ID = "alamcenfordecs_1730138642163"  # @param {type:"string"}
LOCATION_ID = "us"  # @param {type:"string"}

### Define Python functions as tools

The second component of your agent involves Python functions as tools, which will enable the Gemini model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

In this example, you'll define a function that sends a query to Vertex AI Search and returns relevant records from the data store that you created in the previous section:

In [30]:
pip install unstructured "unstructured[pdf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.5/488.5 kB 34.5 MB/s eta 

In [6]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (390 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [8]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,867 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123652 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [9]:
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.2 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Fetched 3,743 kB in 1s (4,079 kB/s)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123699 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubunt

In [11]:
!pip install nltk

In [109]:
pip install langchain_google_firestore

In [114]:
!gcloud config set project "llmcoppelpruebas"

Are you sure you wish to set property [core/project] to llmcoppelpruebas?

Do you want to continue (Y/n)?  y

Updated property [core/project].


In [138]:
# def search_infoIn_decs(query: str) -> str:
#   """Obtiebe la info sobre las decisiones a traves de search retriever.
#     Args:
#         search_query (str): The query to search for.
#     Returns:
#         str: The lists of decs.
#   """
#   from langchain_google_community import VertexAISearchRetriever

#   retriever = VertexAISearchRetriever(
#       project_id=PROJECT_ID,
#       data_store_id=DATA_STORE_ID,
#       location_id=LOCATION_ID,
#       engine_data_type=0,
#       max_documents=1,
#       max_extractive_answer_count=1,
#   )

#   result = str(retriever.invoke(query))
#   return

from typing import List
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine


def search_infoIn_decs(
    search_query: str,
) -> List[discoveryengine.SearchResponse]:
    """Get the information about promotions from Vertex Search.
    Args:
        search_query (str): The query to search for.
    Returns:
        str: The lists of promotions.
    """

    project_id = "llmcoppelpruebas"
    location = "global"
    engine_id = "searchfordec_1730138608816"


    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"



    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        #content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    print(response)

    return response

Now you can test your search function with sample input to ensure that it's working as expected:

In [139]:
search_infoIn_decs("Obten el inicio del texto de la decision 91")

PermissionDenied: 403 Permission denied on resource project llmcoppelpruebas. [links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
, reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/llmcoppelpruebas"
}
metadata {
  key: "service"
  value: "discoveryengine.googleapis.com"
}
]

<div class="alert alert-block alert-warning">
<b>⚠️ Important notes:</b><br>
<br>
<b>If you get an empty response when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;Check the status of your data store in Vertex AI Search and try running the cell again after indexing is complete.<br>
&nbsp;&nbsp;&nbsp;&nbsp;Once you've created your search app and data store, you can view the <b>Activity</b> tab within your data store's <b>Data</b> page to see the status of your data ingestion.<br>
&nbsp;&nbsp;&nbsp;&nbsp;For this dataset, the import and indexing process takes about 5 to 10 minutes.<br>
<br>
<b>If you get an error when running the previous cell:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;In order for this sample notebook to work with the Google Vertex AI Search retriever in LangChain,<br>
&nbsp;&nbsp;&nbsp;&nbsp;you'll need to create a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store">data store</a> <b>and</b> a <a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app">search app</a> associated with it in Vertex AI Search.<br>
&nbsp;&nbsp;&nbsp;&nbsp;If you only create a data store, the LangChain retriever might throw errors when making queries against the data store.
</div>

</div>

### Define agent

The third component of your agent involves adding a reasoning layer, which helps your agent use the tools that you provided to help the end user achieve a higher-level goal.

If you were to use Gemini and Function Calling on their own without a reasoning layer, you would need to handle the process of calling functions and APIs in your application code, and you would need to implement retries and additional logic to ensure that your function calling code is resilient to failures and malformed requests.

Define the prompt template and initialize the chat session history:

In [84]:
# Define prompt template
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Now you'll use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine, which brings together the model, tools, and reasoning that you've built up so far:

In [85]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_infoIn_decs],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

### Test your agent locally

Now you can test the model and agent behavior to ensure that it's working as expected before you deploy it:

In [102]:
response = agent.query(
    input="Quien escribio la decision 21A?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

La decisión 21A no se encuentra en la base de datos. 


### Deploy your agent on Vertex AI

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [69]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_infoIn_decs],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

Now you're ready to deploy your agent to Reasoning Engine in Vertex AI by calling `reasoning_engines.ReasoningEngine.create()` along with the instance of your agent and the Python packages that your agent requires at runtime:

In [77]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "langchain-google-community",
        "google-cloud-discoveryengine",
    ],
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket bucketfordecs
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://bucketfordecs/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://bucketfordecs/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://bucketfordecs/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/51864270156/locations/us-central1/reasoningEngines/2464119907055304704/operations/7382307607907663872
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/51864270156/locations/us-central1/reasoningEngines/2464119907055304704
INFO:vertexai.reasoning_engin

### Grant Discovery Engine Editor access to Reasoning Engine service account

Before you send queries to your remote agent, you'll need to grant the **Discovery Engine Editor** role to the Reasoning Engine service account.

After you've completed this step, you remote agent will be able to retrieve documents from the data store that you created in Vertex AI Search:

In [122]:
# Retrieve the project number associated with your project ID
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

'51864270156'

In [123]:
# Add a new role binding to the IAM policy
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.admin

Updated IAM policy for project [llmcoppelprueba].
bindings:
- members:
  - serviceAccount:service-51864270156@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-51864270156@gcp-sa-aiplatform-re.iam.gserviceaccount.com
  role: roles/aiplatform.reasoningEngineServiceAgent
- members:
  - serviceAccount:service-51864270156@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-51864270156@gcp-sa-discoveryengine.iam.gserviceaccount.com
  role: roles/compute.storageAdmin
- members:
  - serviceAccount:service-51864270156@gcp-sa-dataform.iam.gserviceaccount.com
  role: roles/dataform.serviceAgent
- members:
  - serviceAccount:51864270156-compute@developer.gserviceaccount.com
  - serviceAccount:service-51864270156@gcp-sa-aiplatform-re.iam.gserviceaccount.com
  - serviceAccount:service-51864270156@gcp-sa-discoveryengine.iam.gserviceaccount.com
  - user:jos.

### Test your remotely deployed agent

With all of the core components of your community solar planning agent in place, you can send prompts to your remotely deployed agent to perform different tasks and test that it's working as expected:

In [81]:
response = remote_agent.query(
    input="De que trata la decision 9-B?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

La Decisión 9-B trata sobre el manejo correcto de muebles.  Esta decisión busca concientizar a los colaboradores sobre la importancia de cuidar los muebles para evitar pérdidas y cobros a los responsables del mal manejo.  También incluye recomendaciones para el manejo de la mercancía en los Manuales MC de las diferentes áreas del CEDIS Regional. 


In [53]:
response = remote_agent.query(
    input="Who are the actors in The Matrix?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Here are some of the main actors in The Matrix:

* **Keanu Reeves** as Neo (Thomas Anderson)
* **Carrie-Anne Moss** as Trinity
* **Laurence Fishburne** as Morpheus
* **Hugo Weaving** as Agent Smith
* **Gloria Foster** as The Oracle (in the first film)
* **Jada Pinkett Smith** as Niobe
* **Joe Pantoliano** as Cypher
* **Marcus Chong** as Tank
* **Julian Arahanga** as  Apoc
* **Matt Doran** as Mouse
* **Anthony Wong** as  The Keymaster
* **Lambert Wilson** as The Merovingian (in *The Matrix Reloaded* and *The Matrix Revolutions*)
* **Collin Chou** as Seraph (in *The Matrix Reloaded* and *The Matrix Revolutions*)
* **Harry Lennix** as  Agent  Smith (in *The Matrix Reloaded* and *The Matrix Revolutions*)

This is not an exhaustive list, but it includes some of the most prominent and recognizable actors in the film. 


In [ ]:
response = remote_agent.query(
    input="Are those actors in any other movies?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Yes, all of those actors have been in many other movies! Here are a few examples for each:

* **Keanu Reeves:** John Wick (2014), The Devil's Advocate (1997), Speed (1994), Bill & Ted's Excellent Adventure (1989)
* **Laurence Fishburne:**  Apocalypse Now (1979), Boyz n the Hood (1991), What's Love Got to Do with It (1993), Contagion (2011)
* **Carrie-Anne Moss:** Memento (2000), Chocolat (2000), Disturbia (2007), Jessica Jones (TV series, 2015-2019)
* **Hugo Weaving:** The Lord of the Rings trilogy (2001-2003), V for Vendetta (2005), Captain America: The First Avenger (2011), The Hobbit trilogy (2012-2014)
* **Joe Pantoliano:** The Goonies (1985), Risky Business (1983), The Fugitive (1993), Memento (2000) 


### Reusing your deployed agent from other applications or SDKs

You can now import and use the remotely deployed Reasoning Engine in this notebook session or in a different notebook or Python script by uncommenting and adapting the following code:

In [ ]:
# from vertexai.preview import reasoning_engines

# PROJECT_ID = "YOUR_PROJECT_ID"
# LOCATION = "YOUR_LOCATION"
# REASONING_ENGINE_ID = "YOUR_REASONING_ENGINE_ID"

# remote_agent = reasoning_engines.ReasoningEngine(f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{REASONING_ENGINE_ID}")
# response = remote_agent.query(input=query)

Or, you can query your agent from other programming languages using any of the [available client libraries in Vertex AI](https://cloud.google.com/vertex-ai/docs/start/client-libraries), including C#, Java, Node.js, Python, Go, or REST API.

## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Reasoning Engine instance to avoid any unexpected charges on your Google Cloud account.

In [ ]:
remote_agent.delete()